In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import matplotlib.pyplot as plt

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://www.imdb.com/chart/top?sort=ir,desc&mode=simple&page=1'
browser.visit(url)

In [4]:
# Soupify the web page.

html = browser.html
imdb = BeautifulSoup(html)
type(imdb)

bs4.BeautifulSoup

In [5]:
# Find the table list of top 250 movies.

top_movies = imdb.find("table", {"data-caller-name":"chart-top250movie"}).find("tbody").find_all("tr")
len(top_movies)

250

In [6]:
# Test scrape to find the Movie ID for the first movie in the Top 250 list.

test = top_movies[0]
movie_id1 = test.find("td", {"class": "watchlistColumn"}).find("div")["data-tconst"]
movie_id1

'tt0111161'

In [7]:
# Web scrapping loop to extract movie titles and corresponding movie_id.

top_250_ids = []

for movie in top_movies:
    title = movie.find("td", {"class": "titleColumn"}).find("a").text
    movie_id = movie.find("td", {"class": "watchlistColumn"}).find("div")["data-tconst"]
    
    row = {}
    row["title"] = title
    row["movie_id"] = movie_id
    top_250_ids.append(row)
    
movie_ids_df = pd.DataFrame(top_250_ids)
movie_ids_df

,title,movie_id
0,The Shawshank Redemption,tt0111161
1,The Godfather,tt0068646
2,The Dark Knight,tt0468569
3,The Godfather Part II,tt0071562
4,12 Angry Men,tt0050083
...,...,...
245,Everything Everywhere All at Once,tt6710474
246,The Iron Giant,tt0129167
247,The Help,tt1454029
248,Aladdin,tt0103639


In [8]:
browser.quit()

In [11]:
# Using the OMDB API to acquire movie metadata. Initial test with The Shawshank Redemption movie ID.
